# Deploy ComfyUI on RunPod

This notebook deploys a RunPod instance with ComfyUI using available GPU resources and the ashleykleynhans/comfyui-docker template.

## Prerequisites
- RunPod API key set in `.env` file
- `runpod` Python package installed
- Optional: RunPod Network Volume ID for persistent storage

In [1]:
# Install required packages if not already installed
!pip install runpod python-dotenv ipywidgets tqdm requests

In [2]:
import os
import runpod
import time
import json
import requests
from dotenv import load_dotenv
from datetime import datetime
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

## Configuration and Authentication

In [3]:
# Load environment variables
load_dotenv()

RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY")
if not RUNPOD_API_KEY:
    raise ValueError("RUNPOD_API_KEY environment variable not set. Please set it in your .env file.")

runpod.api_key = RUNPOD_API_KEY
print(f"RunPod API key loaded: {RUNPOD_API_KEY[:16]}...")

RunPod API key loaded: rpa_6R0NGJYCZ99O...


In [4]:
# Configuration for ComfyUI Pod
POD_NAME_PREFIX = "ComfyUI"
COMFYUI_IMAGE = "ashleykleynhans/comfyui:latest"  # Using the recommended ComfyUI Docker template
GPU_COUNT = 1
CONTAINER_DISK_SIZE_GB = 100  # Larger disk for ComfyUI models and workflows
NETWORK_VOLUME_ID = os.getenv("RUNPOD_NETWORK_VOLUME_ID")  # Optional persistent storage

print(f"Docker Image: {COMFYUI_IMAGE}")
print(f"Disk Size: {CONTAINER_DISK_SIZE_GB}GB")
if NETWORK_VOLUME_ID:
    print(f"Network Volume: {NETWORK_VOLUME_ID}")
else:
    print("Network Volume: Not configured (using container storage only)")

Docker Image: ashleykleynhans/comfyui:latest
Disk Size: 100GB
Network Volume: Not configured (using container storage only)


## Available GPU Discovery

In [5]:
def get_available_gpus():
    """Get list of available GPUs from RunPod."""
    try:
        gpus = runpod.get_gpus()
        print("Available GPUs:")
        print("-" * 80)
        
        available_gpus = []
        for gpu in gpus:
            gpu_id = gpu.get('id', 'N/A')
            gpu_name = gpu.get('displayName', 'N/A')
            memory_gb = gpu.get('memoryInGb', 'N/A')
            price_per_hour = gpu.get('lowestPrice', {}).get('minimumBidPrice', 'N/A')
            available_count = gpu.get('lowestPrice', {}).get('uninterruptableCountAvailable', 0)
            
            if available_count > 0:  # Only show available GPUs
                print(f"ID: {gpu_id} | {gpu_name} | {memory_gb}GB | ${price_per_hour}/hr | Available: {available_count}")
                available_gpus.append({
                    'id': gpu_id,
                    'name': gpu_name,
                    'memory': memory_gb,
                    'price': price_per_hour,
                    'available': available_count
                })
        
        print("-" * 80)
        print(f"Found {len(available_gpus)} available GPU types")
        return available_gpus
        
    except Exception as e:
        print(f"Error fetching GPU list: {e}")
        return []

# Get available GPUs
available_gpus = get_available_gpus()

Available GPUs:
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Found 0 available GPU types


## Interactive Configuration

In [ ]:
# Create GPU selection dropdown
if available_gpus:
    gpu_options = [(f"{gpu['name']} ({gpu['memory']}GB) - ${gpu['price']}/hr", gpu['id']) 
                   for gpu in available_gpus]
    
    # Prefer RTX 4090 or RTX 5090 if available, otherwise use first available
    preferred_gpu = None
    for gpu in available_gpus:
        if 'RTX 5090' in gpu['name'] or 'RTX 4090' in gpu['name']:
            preferred_gpu = gpu['id']
            break
    
    default_gpu = preferred_gpu if preferred_gpu else available_gpus[0]['id']
else:
    gpu_options = [("No GPUs available", "")]
    default_gpu = ""

# Interactive widgets for configuration
gpu_widget = widgets.Dropdown(
    options=gpu_options,
    value=default_gpu,
    description='GPU Type:',
    style={'description_width': 'initial'}
)

pod_name_widget = widgets.Text(
    value=f"{POD_NAME_PREFIX}-{int(time.time())}",
    description='Pod Name:',
    style={'description_width': 'initial'}
)

auto_shutdown_widget = widgets.IntSlider(
    value=60,
    min=15,
    max=240,
    step=15,
    description='Auto-shutdown (min):',
    style={'description_width': 'initial'}
)

install_vibeVoice_widget = widgets.Checkbox(
    value=False,
    description='Install VibeVoice workflow on startup',
    style={'description_width': 'initial'}
)

display(widgets.VBox([
    widgets.HTML("<h3>Pod Configuration</h3>"),
    gpu_widget,
    pod_name_widget,
    auto_shutdown_widget,
    install_vibeVoice_widget
]))

## Pod Creation Functions

In [ ]:
def create_comfyui_pod(pod_name, image_name, gpu_type_id, gpu_count, disk_size, 
                       network_volume_id=None, auto_shutdown_min=60, install_vibeVoice=False):
    """
    Creates a RunPod GPU Pod for ComfyUI with specified configurations.
    """
    print(f"Creating RunPod with name: {pod_name} and GPU ID: {gpu_type_id}...")
    
    try:
        # Environment variables for ComfyUI container
        env_vars = {
            "COMFYUI_PORT": "8188",
            "COMFYUI_HOST": "0.0.0.0",
            "AUTO_SHUTDOWN_DELAY": str(auto_shutdown_min),
            "INSTALL_VIBEVOICE": "true" if install_vibeVoice else "false"
        }
        
        # Add HuggingFace token if available
        hf_token = os.getenv("HUGGINGFACE_TOKEN")
        if hf_token:
            env_vars["HUGGINGFACE_TOKEN"] = hf_token
            print("HuggingFace token configured for model downloads")
        
        pod_config = {
            "name": pod_name,
            "image_name": image_name,
            "gpu_type_id": gpu_type_id,
            "gpu_count": gpu_count,
            "volume_in_gb": disk_size,
            "ports": "8188/http,22/tcp",  # ComfyUI web interface and SSH
            "env": env_vars
        }
        
        # Add network volume if specified
        if network_volume_id:
            pod_config["network_volume_id"] = network_volume_id
            pod_config["volume_mount_path"] = "/workspace/models"  # Mount for persistent model storage
            print(f"Mounting network volume {network_volume_id} to /workspace/models")
        
        pod = runpod.create_pod(**pod_config)
        print(f"Pod creation initiated. Pod ID: {pod['id']}")
        return pod
        
    except Exception as e:
        print(f"Error creating pod: {e}")
        return None

In [ ]:
def monitor_pod_startup(pod_id, pod_name, max_wait_minutes=10):
    """
    Monitor pod startup with progress bar and status updates.
    """
    print(f"Monitoring pod startup for '{pod_name}' (ID: {pod_id})...")
    print("This may take a few minutes as the ComfyUI image downloads and initializes.")
    
    max_attempts = max_wait_minutes * 12  # Check every 5 seconds
    
    with tqdm(total=max_attempts, desc="Waiting for pod", unit="checks") as pbar:
        for attempt in range(1, max_attempts + 1):
            time.sleep(5)
            
            try:
                current_pod = runpod.get_pod(pod_id)
                if not current_pod:
                    pbar.set_description("Pod not found")
                    continue
                
                status = current_pod.get('desiredStatus', 'UNKNOWN')
                runtime_status = current_pod.get('runtime', {}).get('uptimeInSeconds', 0)
                
                pbar.set_description(f"Status: {status}")
                pbar.update(1)
                
                if status == 'RUNNING' and runtime_status > 30:  # Give it time to fully initialize
                    pbar.set_description("Pod is running!")
                    return current_pod
                    
            except Exception as e:
                pbar.set_description(f"Error: {str(e)[:30]}...")
                continue
    
    print(f"Pod '{pod_name}' did not reach RUNNING status within {max_wait_minutes} minutes.")
    return None

In [ ]:
def display_pod_info(pod_data):
    """
    Display comprehensive pod information in a formatted way.
    """
    if not pod_data:
        print("No pod data available")
        return
    
    pod_id = pod_data.get('id', 'N/A')
    pod_name = pod_data.get('name', 'N/A')
    public_ip = pod_data.get('publicIp', 'N/A')
    
    # Parse ports information
    ports_info = pod_data.get('ports', {})
    if isinstance(ports_info, str):
        try:
            ports_info = json.loads(ports_info)
        except:
            ports_info = {}
    
    # Extract ComfyUI web interface info
    comfyui_port_info = ports_info.get('8188/http', {})
    comfyui_port = comfyui_port_info.get('publicPort', 'N/A')
    comfyui_url = comfyui_port_info.get('publicUrl', 'N/A')
    
    # Extract SSH info
    ssh_port_info = ports_info.get('22/tcp', {})
    ssh_port = ssh_port_info.get('publicPort', 'N/A')
    
    # Runtime information
    runtime = pod_data.get('runtime', {})
    uptime = runtime.get('uptimeInSeconds', 0)
    gpu_info = runtime.get('gpus', [{}])[0] if runtime.get('gpus') else {}
    gpu_name = gpu_info.get('gpuType', 'N/A')
    
    # Create formatted display
    info_html = f"""
    <div style="border: 2px solid #4CAF50; border-radius: 10px; padding: 20px; margin: 10px 0; background-color: #f9f9f9;">
        <h2 style="color: #4CAF50; margin-top: 0;">ComfyUI Pod Successfully Created</h2>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
            <div>
                <h3 style="color: #333; border-bottom: 2px solid #ddd;">Pod Details</h3>
                <p><strong>Name:</strong> {pod_name}</p>
                <p><strong>ID:</strong> {pod_id}</p>
                <p><strong>Public IP:</strong> {public_ip}</p>
                <p><strong>GPU:</strong> {gpu_name}</p>
                <p><strong>Uptime:</strong> {uptime} seconds</p>
            </div>
            
            <div>
                <h3 style="color: #333; border-bottom: 2px solid #ddd;">Access Information</h3>
                <p><strong>ComfyUI Web Interface:</strong></p>
                <p style="margin-left: 20px;">Port: {comfyui_port}</p>
                <p style="margin-left: 20px;">URL: <a href="{comfyui_url}" target="_blank">{comfyui_url}</a></p>
                <p><strong>SSH Access:</strong></p>
                <p style="margin-left: 20px;">Port: {ssh_port}</p>
                <p style="margin-left: 20px;">Command: <code>ssh root@{public_ip} -p {ssh_port}</code></p>
            </div>
        </div>
        
        <div style="background-color: #e8f5e8; padding: 15px; border-radius: 5px; margin-top: 20px;">
            <h3 style="color: #2e7d32; margin-top: 0;">Next Steps</h3>
            <ol>
                <li>Click the ComfyUI URL above to access the web interface</li>
                <li>Wait for ComfyUI to fully initialize (may take 1-2 minutes)</li>
                <li>Upload your workflows or use the built-in examples</li>
                <li>Remember to terminate the pod when finished to avoid charges</li>
            </ol>
        </div>
    </div>
    """
    
    display(HTML(info_html))
    
    # Also print key info for easy copying
    print(f"\nQuick Access Info:")
    print(f"ComfyUI URL: {comfyui_url}")
    print(f"SSH Command: ssh root@{public_ip} -p {ssh_port}")
    print(f"Pod ID: {pod_id}")

## Deploy ComfyUI Pod

In [ ]:
# Create the ComfyUI pod with current configuration
if not available_gpus:
    print("No GPUs available. Cannot create pod.")
else:
    pod_name = pod_name_widget.value
    gpu_type_id = gpu_widget.value
    auto_shutdown = auto_shutdown_widget.value
    install_vibeVoice = install_vibeVoice_widget.value
    
    # Find GPU name for display
    gpu_name = "Unknown"
    for gpu in available_gpus:
        if gpu['id'] == gpu_type_id:
            gpu_name = gpu['name']
            break
    
    print(f"Creating ComfyUI pod with configuration:")
    print(f"   Name: {pod_name}")
    print(f"   GPU: {gpu_name} (ID: {gpu_type_id})")
    print(f"   Auto-shutdown: {auto_shutdown} minutes")
    print(f"   Install VibeVoice: {install_vibeVoice}")
    print("\n" + "="*50)
    
    # Create the pod
    pod = create_comfyui_pod(
        pod_name=pod_name,
        image_name=COMFYUI_IMAGE,
        gpu_type_id=gpu_type_id,
        gpu_count=GPU_COUNT,
        disk_size=CONTAINER_DISK_SIZE_GB,
        network_volume_id=NETWORK_VOLUME_ID,
        auto_shutdown_min=auto_shutdown,
        install_vibeVoice=install_vibeVoice
    )
    
    if pod:
        pod_id = pod['id']
        print(f"\nPod creation request successful!")
        print(f"Pod ID: {pod_id}")
    else:
        print("\nFailed to create pod. Check your configuration and try again.")

In [ ]:
# Monitor pod startup (only run if pod was created successfully)
if 'pod' in locals() and pod:
    running_pod = monitor_pod_startup(pod['id'], pod_name, max_wait_minutes=15)
    
    if running_pod:
        print("\nPod is now running! Displaying connection information...\n")
        display_pod_info(running_pod)
    else:
        print("\nPod startup monitoring timed out. Check RunPod console for details.")
        print(f"Pod ID: {pod['id']}")
else:
    print("No pod to monitor. Please run the pod creation cell first.")

## Pod Management Functions

In [ ]:
def list_active_pods():
    """
    List all active ComfyUI pods for the current account.
    """
    try:
        pods = runpod.get_pods()
        comfyui_pods = [pod for pod in pods if 'ComfyUI' in pod.get('name', '') and 
                       pod.get('desiredStatus') not in ('TERMINATED', 'DELETED')]
        
        if not comfyui_pods:
            print("No active ComfyUI pods found.")
            return []
        
        print(f"Found {len(comfyui_pods)} active ComfyUI pod(s):\n")
        
        for pod in comfyui_pods:
            name = pod.get('name', 'N/A')
            pod_id = pod.get('id', 'N/A')
            status = pod.get('desiredStatus', 'N/A')
            gpu_type = pod.get('gpuType', 'N/A')
            uptime = pod.get('runtime', {}).get('uptimeInSeconds', 0)
            
            print(f"Pod: {name} (ID: {pod_id})")
            print(f"   Status: {status} | GPU: {gpu_type} | Uptime: {uptime}s")
            print()
        
        return comfyui_pods
        
    except Exception as e:
        print(f"Error listing pods: {e}")
        return []

def terminate_pod(pod_id):
    """
    Terminate a specific pod by ID.
    """
    try:
        result = runpod.terminate_pod(pod_id)
        print(f"Pod {pod_id} termination requested successfully.")
        return True
    except Exception as e:
        print(f"Error terminating pod {pod_id}: {e}")
        return False

# List current active pods
active_pods = list_active_pods()

In [ ]:
# Terminate pod (uncomment and modify pod_id to use)
# CAUTION: This will terminate the pod and you will lose any unsaved work!

# pod_id_to_terminate = "your-pod-id-here"
# terminate_pod(pod_id_to_terminate)

## Health Check and Validation

In [ ]:
def test_comfyui_connection(pod_url):
    """
    Test connection to ComfyUI web interface.
    """
    try:
        # Test basic connectivity
        response = requests.get(f"{pod_url}/", timeout=10)
        if response.status_code == 200:
            print(f"ComfyUI web interface is accessible at {pod_url}")
            
            # Test API endpoint
            api_response = requests.get(f"{pod_url}/system_stats", timeout=5)
            if api_response.status_code == 200:
                print("ComfyUI API is responding")
                return True
            else:
                print("ComfyUI web interface accessible but API may still be initializing")
                return True
        else:
            print(f"ComfyUI returned status code: {response.status_code}")
            return False
            
    except requests.exceptions.RequestException as e:
        print(f"Cannot connect to ComfyUI: {e}")
        print("ComfyUI may still be initializing. Wait a few minutes and try again.")
        return False

# Test connection to the created pod (modify URL as needed)
if 'running_pod' in locals() and running_pod:
    ports_info = running_pod.get('ports', {})
    if isinstance(ports_info, str):
        try:
            ports_info = json.loads(ports_info)
        except:
            ports_info = {}
    
    comfyui_url = ports_info.get('8188/http', {}).get('publicUrl', '')
    if comfyui_url:
        print(f"Testing connection to ComfyUI at {comfyui_url}...")
        test_comfyui_connection(comfyui_url)
    else:
        print("No ComfyUI URL available for testing")
else:
    print("No running pod available for testing. Create a pod first.")

## Summary and Next Steps

This notebook provides a complete workflow for:

1. **Discovering available GPUs** and selecting the best option for your needs
2. **Creating RunPod instances** with ComfyUI pre-installed
3. **Monitoring deployment progress** with real-time status updates
4. **Managing pod lifecycle** including listing and termination
5. **Health checking** to verify ComfyUI is accessible

### Next Steps:
- Use the **deploy-vibeVoice-workflow.ipynb** notebook to install VibeVoice
- Explore the ComfyUI web interface at the provided URL
- Upload your own workflows or use built-in examples
- **Remember to terminate pods** when finished to avoid unnecessary charges

### Tips:
- ComfyUI may take 1-2 minutes to fully initialize after the pod starts
- Use network volumes for persistent model storage across sessions
- Monitor your RunPod usage and costs in the RunPod console
- Keep your API keys secure and never commit them to version control